In [ ]:
!conda install -y -c conda-forge transformers datasets seqeval

In [ ]:
!pip install transformers datasets seqeval

In [1]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

Import libraries

In [5]:
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification
)
from datasets import Dataset
import numpy as np
import json
from sklearn.metrics import classification_report

Load training and evaluation data

In [6]:
# Upload these files or make sure they're in your current working directory
with open("train_data.json") as f:
    train_data = json.load(f)

with open("eval_data.json") as f:
    eval_data = json.load(f)


In [7]:
# Extract all unique labels
unique_labels = sorted(list({label for sample in train_data for label in sample['labels']}))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}


In [8]:
def encode_labels(example):
    return {
        "tokens": example["tokens"],
        "labels": [label2id[label] for label in example["labels"]]
    }

train_dataset = list(map(encode_labels, train_data))
eval_dataset = list(map(encode_labels, eval_data))


In [9]:
from datasets import Dataset

train_dataset = Dataset.from_list(train_dataset)
eval_dataset = Dataset.from_list(eval_dataset)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
eval_dataset = eval_dataset.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 191/191 [00:00<00:00, 1225.76 examples/s]


In [ ]:
!conda install -y pytorch torchvision torchaudio cpuonly -c pytorch

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-radiology-token-classifier",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    do_eval=True,           # Enable evaluation
    save_steps=500          # Optional: how often to save
)



In [17]:
from transformers import DataCollatorForTokenClassification, Trainer
from sklearn.metrics import classification_report

# Create a data collator that will pad your inputs and labels to the longest sequence in a batch
data_collator = DataCollatorForTokenClassification(tokenizer)

# Define compute_metrics with flattened label sequences
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = []
    true_preds = []

    for pred_seq, label_seq in zip(predictions, labels):
        for pred, label in zip(pred_seq, label_seq):
            if label != -100:
                true_labels.append(label)
                true_preds.append(pred)

    report = classification_report(true_labels, true_preds, output_dict=True)

    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"]
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/var/folders/97/tj5167w92cqbx7z6t1shzr4w0000gn/T/ipykernel_6892/786441743.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

/opt/anaconda3/envs/nlp_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
10,1.936200
20,1.189500
30,0.766900
40,0.484300
50,0.336500
60,0.306500
70,0.263200
80,0.166400
90,0.279100
100,0.166100


TrainOutput(global_step=288, training_loss=0.25484322301215595, metrics={'train_runtime': 6737.6239, 'train_samples_per_second': 0.34, 'train_steps_per_second': 0.043, 'total_flos': 251362365195270.0, 'train_loss': 0.25484322301215595, 'epoch': 3.0})

In [18]:
results = trainer.evaluate()
print("Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")

/opt/anaconda3/envs/nlp_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Evaluation Results:
eval_loss: 0.052343737334012985
eval_model_preparation_time: 0.009
eval_precision: 0.987212377109425
eval_recall: 0.9871664733178654
eval_f1: 0.9871175118005348
eval_runtime: 54.4215
eval_samples_per_second: 3.51
eval_steps_per_second: 0.441


In [19]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)

In [11]:
import sys
print(sys.executable)

/opt/anaconda3/envs/nlp_env/bin/python


In [10]:
import transformers
print(transformers.__version__)


4.54.1


In [20]:
# Decode a sample input
sample = eval_dataset[0]
tokens = tokenizer.convert_ids_to_tokens(sample["input_ids"])
print(tokens)


['[CLS]', 'CH', '##ES', '##T', 'PA', 'AND', 'LA', '##TE', '##RA', '##L', 'C', '##L', '##IN', '##IC', '##AL', 'IN', '##F', '##OR', '##MA', '##TI', '##ON', ':', 'Ch', '##est', 'tight', '##ness', 'and', 'short', '##ness', 'of', 'breath', 'today', '.', 'CO', '##MP', '##AR', '##IS', '##ON', ':', 'CH', '##ES', '##T', 'PA', 'dated', '11', '/', '10', '/', '2008', 'F', '##IN', '##DI', '##NG', '##S', ':', 'Heart', ',', 'lungs', 'and', 'vessels', 'normal', '.', 'No', 'p', '##ne', '##um', '##oth', '##orax', ',', 'p', '##le', '##ural', 'e', '##ff', '##usion', 'or', 'ad', '##eno', '##pathy', '.', 'No', 'significant', 'bone', 'abnormal', '##ity', '.', 'I', '##MP', '##RE', '##SS', '##ION', ':', 'N', '##eg', '##ative', 'chest', '.', 'Sign', '##ed', 'by', ':', '[', '[', 'P', '##ER', '##SO', '##NA', '##L', '##NA', '##ME', ']', ']', '[SEP]']


In [21]:
import torch

def predict_labels(model, tokenizer, input_text):
    model.eval()
    tokens = tokenizer(input_text, return_tensors="pt", truncation=True, is_split_into_words=False)
    with torch.no_grad():
        outputs = model(**tokens)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)
    predicted_labels = [model.config.id2label[label_id.item()] for label_id in predictions[0]]
    tokens_decoded = tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])
    
    return list(zip(tokens_decoded, predicted_labels))


In [22]:
from transformers import AutoModelForTokenClassification

base_model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label2id))
base_model.config.id2label = id2label
base_model.config.label2id = label2id


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from transformers import AutoModelForTokenClassification

fine_tuned_model = AutoModelForTokenClassification.from_pretrained("./fine_tuned_model")


In [24]:
sample_text = "EXAM: CHEST CT. FINDINGS: No acute disease. IMPRESSION: Normal study."

print("\n🔹 Before Fine-Tuning:")
print(predict_labels(base_model, tokenizer, sample_text))

print("\n🔹 After Fine-Tuning:")
print(predict_labels(fine_tuned_model, tokenizer, sample_text))



🔹 Before Fine-Tuning:


/opt/anaconda3/envs/nlp_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[('[CLS]', 'I-Findings'), ('E', 'O'), ('##X', 'B-Findings'), ('##AM', 'E-Clinical_History'), (':', 'B-Exam_Name_and_Date'), ('CH', 'I-Findings'), ('##ES', 'O'), ('##T', 'O'), ('CT', 'I-Findings'), ('.', 'B-Exam_Name_and_Date'), ('F', 'O'), ('##IN', 'E-Impression'), ('##DI', 'O'), ('##NG', 'I-Findings'), ('##S', 'O'), (':', 'B-Exam_Name_and_Date'), ('No', 'I-Findings'), ('acute', 'I-Findings'), ('disease', 'I-Findings'), ('.', 'B-Exam_Name_and_Date'), ('I', 'I-Findings'), ('##MP', 'B-Clinical_History'), ('##RE', 'I-Findings'), ('##SS', 'I-Findings'), ('##ION', 'O'), (':', 'B-Exam_Name_and_Date'), ('Normal', 'I-Findings'), ('study', 'I-Findings'), ('.', 'B-Exam_Name_and_Date'), ('[SEP]', 'E-Exam_Name_and_Date')]

🔹 After Fine-Tuning:
[('[CLS]', 'E-Impression'), ('E', 'O'), ('##X', 'O'), ('##AM', 'O'), (':', 'O'), ('CH', 'O'), ('##ES', 'O'), ('##T', 'O'), ('CT', 'O'), ('.', 'O'), ('F', 'B-Findings'), ('##IN', 'B-Findings'), ('##DI', 'B-Findings'), ('##NG', 'B-Findings'), ('##S', 'B-Findin

## 🔍 Before vs After Fine-Tuning: Sample Prediction Comparison

We compare predictions on the following radiology sentence:

**Input:**
> "EXAM: CHEST CT. FINDINGS: No acute disease. IMPRESSION: Normal study."

### 🔹 Before Fine-Tuning:
The model incorrectly labeled many tokens, especially:
- Fragmented token sequences (`EXAM` split into `E`, `##X`, `##AM`) misclassified
- Misaligned `FINDINGS`, `IMPRESSION`, and `EXAM` tokens
- Inconsistent and noisy span boundaries

### 🔹 After Fine-Tuning:
The model shows:
- Accurate segmentation of key medical sections like `FINDINGS` and `IMPRESSION`
- Proper use of BIOES tags (e.g., `B-Findings`, `I-Findings`, `E-Findings`)
- Significantly more structured and interpretable output

This demonstrates the effectiveness of domain-specific fine-tuning for clinical token classification.
